In [ ]:
from numpy import zeros, ones
from numpy.random import randn, randint

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
from tensorflow.keras.utils import plot_model

from matplotlib import pyplot as plt

In [ ]:
import tensorflow as tf
tf.__version__

'2.9.1'

In [ ]:
def define_discriminator(in_shape=(128,128,3)):
	model = Sequential()
	model.add(Conv2D(128, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))

	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))

	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [ ]:
test_discr = define_discriminator()
print(test_discr.summary())

Metal device set to: Apple M1 Pro
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 128)     3584      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128, 128, 128)     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 64, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       147584    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_3 (Conv2D)    

2022-08-01 03:49:25.878100: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-01 03:49:25.878707: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/shikhargupta/opt/miniconda3/envs/GAN/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
def define_generator(latent_dim):
	model = Sequential()

	n_nodes = 128 * 8 * 8 
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((8, 8, 128)))

	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	model.add(Conv2D(3, (8,8), activation='tanh', padding='same')) 
	return model

In [ ]:
test_gen = define_generator(100)
print(test_gen.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 8192)              827392    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 8192)              0         
                                                                 
 reshape (Reshape)           (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 128)      262272    
 nspose)                                                         
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 128)      262272    
 ranspose)                                            

In [ ]:
def define_gan(g_model, d_model):
	d_model.trainable = False
	model = Sequential()
 
	model.add(g_model)
	model.add(d_model)

	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [ ]:
test_gan = define_gan(test_gen, test_discr)
print(test_gan.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 128, 128, 3)       1901059   
                                                                 
 sequential (Sequential)     (None, 1)                 602113    
                                                                 
Total params: 2,503,172
Trainable params: 1,901,059
Non-trainable params: 602,113
_________________________________________________________________
None


In [ ]:
def generate_real_samples(dataset, n_samples):
	ix = randint(0, dataset.shape[0], n_samples)
	X = dataset[ix]
	y = ones((n_samples, 1))
	return X, y

In [ ]:
def generate_latent_points(latent_dim, n_samples):
	x_input = randn(latent_dim * n_samples)
	x_input = x_input.reshape(n_samples, latent_dim) 
	return x_input

In [ ]:
def generate_fake_samples(g_model, latent_dim, n_samples):
	x_input = generate_latent_points(latent_dim, n_samples) 
	X = g_model.predict(x_input)
	y = zeros((n_samples, 1)) 
	return X, y

In [ ]:
def save_plot(examples, epoch, n=10):
	examples = (examples + 1) / 2.0
	for i in range(n * n):
		plt.subplot(n, n, 1 + i)
		plt.axis('off')
		plt.imshow(examples[i])
	filename = 'saved_data_during_training/images/generated_plot_128x128_e%03d.png' % (epoch+1)
	plt.savefig(filename)
	plt.close()

In [ ]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
	X_real, y_real = generate_real_samples(dataset, n_samples)
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	save_plot(x_fake, epoch)
	filename = 'saved_data_during_training/models/generator_model_128x128_%03d.h5' % (epoch+1)
	g_model.save(filename)


In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2) 
	for i in range(n_epochs):
		for j in range(bat_per_epo):
			X_real, y_real = generate_real_samples(dataset, half_batch)
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			X_gan = generate_latent_points(latent_dim, n_batch)
			y_gan = ones((n_batch, 1))
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			print('Epoch>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		if (i+1) % 1 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
import random

In [ ]:
n=9780
SIZE = 128
all_img_list = os.listdir('UTKFace') 

dataset_list = random.sample(all_img_list, n) 

dataset = []
loaded = 0
for img in all_img_list:
    try:
        temp_img = cv2.imread("UTKFace/" + img)
        temp_img = cv2.cvtColor(temp_img, cv2.COLOR_BGR2RGB) 
        temp_img = Image.fromarray(temp_img)
        temp_img = temp_img.resize((SIZE, SIZE)) 
        dataset.append(np.array(temp_img)) 
    except: 
        pass
    loaded += 1
    if loaded % 2500 == 0:
        print('Loaded', loaded, 'images')


Loaded 2500 images
Loaded 5000 images
Loaded 7500 images
Loaded 10000 images
Loaded 12500 images
Loaded 15000 images
Loaded 17500 images
Loaded 20000 images
Loaded 22500 images


In [ ]:
dataset = np.array(dataset) 

dataset = dataset.astype('float32')
dataset = (dataset - 127.5) / 127.5

latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)

train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=250)

2022-08-01 03:50:00.571076: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-01 03:50:00.571636: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-01 03:50:01.975317: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 177ms/step


2022-08-01 03:50:03.722574: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch>1, 1/185, d1=0.682, d2=0.696 g=0.692
2/2 [==============================] - 0s 287ms/step
Epoch>1, 2/185, d1=0.598, d2=0.697 g=0.691
2/2 [==============================] - 0s 287ms/step
Epoch>1, 3/185, d1=0.479, d2=0.700 g=0.689
2/2 [==============================] - 0s 285ms/step
Epoch>1, 4/185, d1=0.294, d2=0.710 g=0.684
2/2 [==============================] - 0s 289ms/step
Epoch>1, 5/185, d1=0.113, d2=0.735 g=0.672
2/2 [==============================] - 0s 289ms/step
Epoch>1, 6/185, d1=0.033, d2=0.771 g=0.668
2/2 [==============================] - 0s 294ms/step
Epoch>1, 7/185, d1=0.015, d2=0.775 g=0.687
2/2 [==============================] - 0s 287ms/step
Epoch>1, 8/185, d1=0.020, d2=0.711 g=0.718
2/2 [==============================] - 0s 288ms/step
Epoch>1, 9/185, d1=0.025, d2=0.669 g=0.762
2/2 [==============================] - 0s 288ms/step
Epoch>1, 10/185, d1=0.014, d2=0.641 g=0.814
2/2 [==============================] - 0s 291ms/step
Epoch>1, 11/185, d1=0.007, d2=0.616 g=0